Description: The purpose of this script is to compare the .html files produced by fmriprep in the output directory to the participant IDs listed in the participants.tsv file in the input (BIDS) directory. If there is NO corresponding .html for the participant ID, fmriprep did not initialize. If there IS a corresponding .html for the participant ID, fmriprep initialized but may not have completed. Further examination is needed to determine if fmriprep was completed (examine .out files) or if uninitialized participants can be processed at a later time (see BIDS count, diagnose errors).

In [6]:
#imports
import argparse
import os
import glob
import pandas as pd
import numpy as np

In [7]:
#to run .py from command line

#parser = argparse.ArgumentParser(description='Sorts a BIDS data directory and give a summary of T1w, fieldmap, and fMRI data.')

#parser.add_argument('--bids_dir', help='Path to the bids directory', type=str)
#parser.add_argument('--out_dir', help='Out put path where you want the summary tables to be saved', type=str)
                    
#args = parser.parse_args()
#bids_dir = args.bids_dir
#out_dir = args.out_dir

In [8]:
#define directories for testing

p_tsv = pd.read_table('/cifs/butler/HBN_data/TD_test_set/participants.tsv') #participants.tsv as dataframe
fmriprep_out = '/cifs/butler/HBN_data/TD_test_set_output' #fmriprep output .htmls
pp_out = '/cifs/butler/HBN_data/preprocessing/postprocessing_outputs' #save summary .csv here


In [9]:
#define functions

def list_files(path, *keywords): #keyword is type of file (e.g., ".html")
    os.chdir(path)
    files = []
    for keyword in keywords:
        files.extend(glob.glob(keyword))
    return [os.path.basename(file) for file in files] #returns list of files in given directory


In [12]:
#compare files

htmls = list_files(fmriprep_out, "*.html*") #gets list of .html files in fmriprep output directory
htmls = [html.strip(".html") for html in htmls] #remove .html from filenames to compare with .tsv 

matches = p_tsv.copy()
matches['html'] = np.where(matches['participant_id'].isin(htmls), 'yes', 'no')

In [11]:
#save out html matches

matches.to_csv(pp_out+'/html-check_all.csv', sep=',', index=False)

#get participants with no .html
no_html = matches[matches["html"] == "no"]
no_html.to_csv(pp_out+'/html-check_no-html.csv', sep=',', index=False)

#get participants with .html
yes_html = matches[matches["html"] == "yes"]
yes_html.to_csv(pp_out+'/html-check_yes-html.csv', sep=',', index=False)
